In [15]:
import datetime
import pandas as pd
from datetime import datetime
import numpy as np
import pyodbc
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.python.framework.graph_util import convert_variables_to_constants
from keras import backend
import tensorflow as tf

In [2]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [3]:
def load_data(x_file, y_file):
    x = pd.read_csv(x_file
                ,header='infer'
                ,delimiter='|'
                ,dtype={
                    'index': 'int'
                    ,'height':'int'
                    ,'width':'int'
                    ,'length':'int'
                    ,'weight':'float'
                    ,'distance':'float'
                })
    y = pd.read_csv(y_file
                ,header='infer'
                ,delimiter='|'
                ,dtype={
                    'index': 'int'
                    ,'usps_priority':'int'
                    ,'fedex_smart': 'int'
                    ,'fedex_ground': 'int'
                })
    
    return x[['height', 'width', 'length', 'weight', 'distance']].values, \
            y[['usps_priority', 'fedex_smart', 'fedex_ground']].values

In [5]:
#model = load_model('rating_pred.h5')
#model = None

In [8]:
x, y = load_data(x_file='ship_x.csv', y_file='ship_y.csv')

print('x.shape =',x.shape)
print('y.shape =',y.shape)
#print(model.evaluate(x, y, verbose=0))

x.shape = (20163, 5)
y.shape = (20163, 3)


In [164]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3,random_state=1100)

In [165]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(209, 5)
(90, 5)
(209, 3)
(90, 3)


In [148]:
early_stopper = EarlyStopping(patience=5)

In [150]:
if not model:
    print('Creating new Model')
    #{'nb_layers': 3, 'optimizer': 'adam', 'nb_neurons': [4, 8, 16], 'activation': ['elu', 'relu', 'tanh']}
    #{'nb_layers': 3, 'optimizer': 'adagrad', 'nb_neurons': [128, 512, 128], 'activation': ['sigmoid', 'sigmoid', 'tanh']}
    #model = Sequential()
    #model.add(Dense(128, activation='sigmoid', input_shape=x_train.shape[1:]))
    #model.add(Dense(512, activation='sigmoid'))
    #model.add(Dense(128, activation='tanh'))
    #model.add(Dense(3, activation='softmax'))

    #{'nb_layers': 3, 'optimizer': 'adagrad', 'nb_neurons': [128, 128, 16], 'activation': ['tanh', 'elu', 'relu']} - Acc. - 79%
    model = Sequential()
    model.add(Dense(128, activation='tanh', input_shape=x_train.shape[1:]))
    model.add(Dense(128, activation='elu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

Creating new Model


In [155]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               768       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 51        
Total params: 19,395
Trainable params: 19,395
Non-trainable params: 0
_________________________________________________________________


In [166]:
model.fit(x_train, y_train,
          batch_size=5,
          epochs=10000,  # using early stopping, so no real limit
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[early_stopper])

Train on 209 samples, validate on 90 samples
Epoch 1/10000
209/209 [==============================] - 0s 347us/step - loss: 0.7646 - acc: 0.7033 - val_loss: 0.6029 - val_acc: 0.7778
Epoch 2/10000
209/209 [==============================] - 0s 342us/step - loss: 0.7298 - acc: 0.7464 - val_loss: 0.6033 - val_acc: 0.7556
Epoch 3/10000
209/209 [==============================] - 0s 259us/step - loss: 0.7086 - acc: 0.7225 - val_loss: 0.5880 - val_acc: 0.7667
Epoch 4/10000
209/209 [==============================] - 0s 259us/step - loss: 0.6936 - acc: 0.7560 - val_loss: 0.6269 - val_acc: 0.7556
Epoch 5/10000
209/209 [==============================] - 0s 238us/step - loss: 0.6841 - acc: 0.7273 - val_loss: 0.5768 - val_acc: 0.7889
Epoch 6/10000
209/209 [==============================] - 0s 216us/step - loss: 0.6781 - acc: 0.7368 - val_loss: 0.5815 - val_acc: 0.7889
Epoch 7/10000
209/209 [==============================] - 0s 200us/step - loss: 0.6727 - acc: 0.7273 - val_loss: 0.5964 - val_acc: 0.8

In [167]:
model.evaluate(x, y, verbose=1)

299/299 [==============================] - 0s 26us/step


[0.5990823466442899, 0.755852842610018]

In [9]:
# height, width, length, weight, distance
model.predict(np.asarray([[4,10,12,2.15,888.1042500059176]]))

array([[0.08316485, 0.77610517, 0.14073005]], dtype=float32)

In [131]:
model.save('rating_pred.h5')

In [132]:
frozen_graph = freeze_session(backend.get_session(), output_names=[out.op.name for out in model.outputs])

INFO:tensorflow:Froze 76 variables.
INFO:tensorflow:Converted 76 variables to const ops.


In [133]:
tf.train.write_graph(frozen_graph, "./", "rating_pred.pb", as_text=False)

'./rating_pred.pb'

In [54]:
date_parser = pd.core.tools.datetimes.to_datetime

In [87]:
gas_price = pd.read_csv('gas_prices.csv'
                 , header='infer'
                 , delimiter=','
                 , dtype={'date': 'str'
                          , 'price': 'float'
                             }
                        , parse_dates=['date']
                        , date_parser=date_parser)

In [88]:
gas_price.iloc[0]['date'].month

1

In [89]:
j = date_parser('6/21/2019')

In [91]:
#gas_price[(gas_price.year <= 2019) & (gas_price.month <= 1)].sort_values(by=['year','month'], ascending=False).head(10)

In [92]:
gas_price.date[0] <= j

True

In [93]:
gas_price[gas_price.date <= j].sort_values(by=['date'], ascending=False).head(1)

date  price
23 2018-12-01  2.457

In [99]:
print("testing %i" % 20)

testing 20


In [19]:
def get_shipping_data(start_date, end_date):
    """Gets Shipping data from HLShipper"""

    print('get_shipping_data :: retrieving data')

    server = 'tcp:hlsqlapp12.hobbylobby.corp'
    database = 'HLShipper'
    connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=_server_;DATABASE=_database_;' \
                        'Trusted_Connection=yes;'
    connection_string = connection_string.replace('_server_', server)
    connection_string = connection_string.replace('_database_', database)

    cnxn = pyodbc.connect(connection_string)
    sql = "SELECT s.Msn, s.Ship_Dttm, s.Total_Charges, s.Shipper_Reference, sp.TrackingNumber_1, " \
          "sp.TrackingNumber_2, sp.Height, sp.Width, sp.Length, sp.Weight, c.Carrier_Name, " \
          "se.Service_Id, se.Service_Symbol, na.city, na.state_province, " \
          "CASE " \
          "WHEN CHARINDEX('-',na.postal_code) > 0 THEN LEFT(na.postal_code,CHARINDEX('-',na.postal_code)-1) " \
          "WHEN LEN(na.postal_code) > 5 THEN LEFT(na.postal_code,5) " \
          "WHEN CHARINDEX('-',na.postal_code) = 0 THEN na.postal_code " \
          "END as postal_code " \
          "FROM [ara].[Shipment] as s " \
          "LEFT JOIN [ara].[Shipment_Package] as sp on s.Msn = sp.Msn " \
          "LEFT JOIN [ara].[Name_Address] as na on s.Name_Address_Id = na.id " \
          "LEFT JOIN [dbo].[Service] as se on s.Service_Id = se.Service_Id " \
          "LEFT JOIN [dbo].[Carrier] as c on se.Carrier_Id = c.Carrier_Id " \
          "WHERE s.Is_Voided_Flag = 0 " \
          "AND s.Shipped_By_User_Id = 'panda' " \
          "AND se.Service_Symbol in ('CONNECTSHIP_ENDICIA.USPS.PRIORITY','FEX.SMART_POST','FEX.FEDEX_GROUND')" \
          "AND s.Ship_Dttm > 'start_date' " \
          "AND s.Ship_Dttm <= 'end_date'" \
          "ORDER BY s.Msn DESC;"

    sql = sql.replace('start_date', start_date)
    sql = sql.replace('end_date', end_date)

    df = pd.read_sql(sql, cnxn)

    print('get_shipping_data :: Records(' + str(len(df)) + ')')

    return df

In [20]:
df = get_shipping_data(start_date="1/10/2019", end_date="1/20/2019")

get_shipping_data :: retrieving data
get_shipping_data :: Records(31201)


In [41]:
df["Ship_Dttm"][0].year - 1

2018